In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
sys.path.append("C:/Users/zih028/Documents/GitHub/rl_rebalance")
from src.baselines import *
from src.targets import *
from src.TilesQ.TilesQ import compute_minimum_variance_weights

In [2]:
def build_minvar_targets_for_period(
    returns: pd.DataFrame,
    cov_dict: dict,
) -> pd.DataFrame:
    """
    For each date in returns, compute min-var weights from cov_dict.
    Falls back to equal-weight if no covariance available that day.
    """
    dates = returns.index
    n_assets = returns.shape[1]
    weights_list = []

    for date in dates:
        if date in cov_dict:
            Sigma = cov_dict[date]
            w = compute_minimum_variance_weights(Sigma)
        else:
            w = np.ones(n_assets) / n_assets
        weights_list.append(w)

    targets = pd.DataFrame(weights_list, index=dates, columns=returns.columns)
    return targets

In [3]:
returns = pd.read_parquet("../data/returns.parquet")
prices = pd.read_parquet("../data/prices.parquet")

with open("../data/cov_oas_window252.pkl", "rb") as f:
    cov_dict = pickle.load(f)

train_start = "2010-01-01"
train_end = "2019-12-31"
test_start = "2020-01-01"
test_end = returns.index.max().strftime("%Y-%m-%d")

returns_train = returns.loc[train_start:train_end]
returns_test = returns.loc[test_start:test_end]

targets_train = build_minvar_targets_for_period(returns_train, cov_dict)
targets_test = build_minvar_targets_for_period(returns_test, cov_dict)

period_M_train = simulate_periodic(
    returns=returns_train,
    targets=targets_train,
    rebalance_freq="M",
    tc_rate=0.001
)

period_M_test = simulate_periodic(
    returns=returns_test,
    targets=targets_test,
    rebalance_freq="M",
    tc_rate=0.001
)

res_band_2_train = simulate_band(
    returns=returns_train,
    targets=targets_train,
    band=0.02,
    tc_rate=0.001
)

res_band_2_test = simulate_band(
    returns=returns_test,
    targets=targets_test,
    band=0.02,
    tc_rate=0.001
)

res_band_4_train = simulate_band(
    returns=returns_train,
    targets=targets_train,
    band=0.04,
    tc_rate=0.001
)

res_band_4_test = simulate_band(
    returns=returns_test,
    targets=targets_test,
    band=0.04,
    tc_rate=0.001
)


C:\Users/zih028/Documents/GitHub/rl_rebalance\src\baselines.py:25: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  rebal_dates = pd.Series(1, index=dates).resample(rebalance_freq).first().index
C:\Users/zih028/Documents/GitHub/rl_rebalance\src\baselines.py:25: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  rebal_dates = pd.Series(1, index=dates).resample(rebalance_freq).first().index


In [5]:
period_M_train.to_csv("../model/baseline_monthly_train.csv")
period_M_test.to_csv("../model/baseline_monthly_test.csv")
res_band_2_train.to_csv("../model/baseline_band2_train.csv")
res_band_2_test.to_csv("../model/baseline_band2_test.csv")
res_band_4_train.to_csv("../model/baseline_band4_train.csv")
res_band_4_test.to_csv("../model/baseline_band4_test.csv")